<a href="https://colab.research.google.com/github/B21-CAP0133/verify-android-app/blob/ML/ML-dir/02_VERIFY_Data_Cleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hari 4

Melakukan data cleansing terhadap verify_dataset yang terdiri atas:

1. One Hot Encoding.
2. Case folding.
3. Menghilangkan kata-kata di dalam kurung.
4. Menghilangkan _punctuation_.
5. Menghilangkan stopwords.
6. Melakukan _stemming_.

Melakukan import library yang dibutuhkan.

In [ ]:
!pip install Sastrawi

In [ ]:
import pandas as pd
import re
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
df = pd.read_csv('verify_dataset.csv')

df.sample(10)

,Judul,Label
1939,Video Cari sensasi biar makin terkenal oleh @O...,1
38,Tulisan Lafadz Allah di Tisu Toilet,HOAX
549,"VIRAL FOTO TEBING ULUWATU BALI RETAK, PIHAK P...",BENAR
2183,Bu Risma minta didoakan agar Surabaya tdk hancur,1
340,“Khoirul Anwar Penemu 4G LTE”,HOAX
1266,"Mulai Tahun Depan, PDIP Minta MUI Agar Babi Pa...",0
871,NU Mendukung LGBT,0
2349,"Tolak Jabatan Menteri, Sandiaga Katakan Lebih ...",1
1919,Omongan Bin Masak Ngak DIperhatikan,1
806,"“Xiami, Jualan HP Atau Komunisme”",BENAR


 ## One Hot Encoding

In [ ]:
for label in sorted(df['Label'].unique()):
  print(label)

0
1
BENAR
HOAX


Pada tahap akusisi dan eksplorasi data, telah diketahui bahwa perlu dilakukan penyeragaman dalam pembuatan label. Maka perlu dilakukan one hot encoding dimana label HOAX akan dijadikan 1 dan label BENAR akan dijadikan 0.

Membuat fungsi one_hot_encoding.

In [ ]:
def one_hot_encoding(label):
  if label == 'BENAR' or label == '0':
    label = 0
  if label == 'HOAX' or label == '1':
    label = 1
  
  return label

Mengaplikasikan fungsi one_hot_encoding kepada dataset.

In [ ]:
df['Label'] = df['Label'].apply(one_hot_encoding)

df.sample(10)

,Judul,Label
78,Nama Jokowi Masuk Daftar Klien Konsultan Asing...,1
1144,KPK Tidak Pinjam Uang Rp. 5 miliar ke Probosut...,0
1717,MOTOR TERBAKAR KARENA HP DIDALAM JOK,1
787,“Artis Dorce Gamalama Tidak Meninggal Dunia”,0
584,Klarifikasi RSUD Tarakan Terkait Kabar Penaha...,0
223,DITUDING TAWARKAN PEKERJAAN KE PERETAS SITUS K...,0
1087,: Dimana TNI dan POLRI dalam Isu Komunisme ata...,0
2259,Pemusnahan babi massal di Thailand,1
1017,Malih Tong-tong Meninggal Dunia,0
1918,Wearing a mask can cause pleurisy,1


Mengecek jumlah setiap label yang ada dalam dataset.

In [ ]:
label_dict = {}
for label in sorted(df['Label']):
  if label not in label_dict:
    label_dict[label] = 0
  label_dict[label] += 1

label_df = pd.DataFrame(label_dict.items(), columns=['Label', 'Count'])
label_df.sort_values(by='Count', ascending=False).head()
#label_df.plot.bar('Label', 'Count')

,Label,Count
0,0,1202
1,1,1164


## Text Cleaning

Text cleaning terdiri atas data cleansing 2, 3, dan 4. Ketiga tahap tersebut dipilih berdasarkan asesmen yang dilakukan saat melakukan eksplorasi data.

Membuat fungsi text_cleaning.

In [ ]:
def text_cleaning(judul):
  judul = judul.lower()                                               # casefolding
  judul = re.sub('\(.*?\) | \[.*?\]', '', judul)                      # kata di dalam kurung
  judul = re.sub('[%s]' % re.escape(string.punctuation), '', judul)   # punctuation
  judul = re.sub('[‘’“”…]', '', judul)                                # tanda kutip

  return judul

Mengaplikasikan fungsi text_cleaning ke dalam dataset.

In [ ]:
df['Judul'] = df['Judul'].apply(text_cleaning)

df.sample(10)

,Judul,Label
172,samsul bahri ketua kpps sydney pro 02,1
2365,starbucks haram di malaysia,1
1824,minyak goreng asri produksi leong hup terbuat ...,1
1305,ini penjelasan mentan soal larangan rockmelon ...,0
1588,lurah pabean tak mengenakan atribut pdip di ra...,0
987,dituding menculik dan berat sebelah pada kasus...,0
1033,bukankah zakat itu satu tahun sekali jadi kal...,0
925,klarifikasi kementerian kelautan dan perikanan...,0
437,pakai menyamar jadi lelaki gelar habib,1
1283,the end of the legend kopi liong bulan,0


## Stopwords

Tahap ini berarti menghilangkan kata-kata yang sering muncul dan tidak terlalu berpengaruh kepada makna kalimat. Tahap ini menggunakan library sastrawi.

Load stopword dari sastrawi.

In [ ]:
factory = StopWordRemoverFactory()
new_stopwords = ['yg', 'dgn', 'dlm', 'nya']
stopwords = factory.get_stop_words() + new_stopwords
stopwords = factory.create_stop_word_remover()

#for stopword in stopwords:
#  print(stopword)

Membuat fungsi stopwords_removal.

In [ ]:
def stopwords_removal(judul):
  judul = stopwords.remove(judul)

  return judul

Mengaplikasikan fungsi stopwords_removal kepada dataset.

In [ ]:
df['Judul'] = df['Judul'].apply(stopwords_removal)

df.sample(10)

,Judul,Label
2316,uang bayaran tim basukidjarot,1
450,petik buah rambutan gratis kandang harimau keb...,1
1665,sby telepon moeldoko suruh jokowi akui kekalahan,1
1352,menanamkan mental paranoid melalui berita hoax,0
871,nu mendukung lgbt,0
1205,klarifikasi dahnil anzar terkait email skenari...,0
1273,rs wates pasang spanduk umumkan bpjs menunggak...,0
594,klarifikasi dahnil anzar terkait email skenar...,0
2289,polling membangun kuil masjid atas tapak masji...,1
1039,pentingnya membaca seluruh isi berita bukan ju...,0


## Steming

Tahap steming merupakan penghapusan imbuhan dalam suatu kata. Tahap ini juga menggunakan library sastrawi.

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

Mengaplikasikan stemmer ke dalam dataset.

In [ ]:
df['Judul'] = df['Judul'].apply(lambda judul:stemmer.stem(judul))

df.sample(10)

,Judul,Label
417,klarifikasi pt kai atas tuduh batal tiket romb...,0
2172,markas indonesia ajar jadi basecamp anies pilk...,1
2044,paham politik gak sih pak,1
137,jokowi sdh kunjung christchurch,1
1684,judul berita tempo mui minta jangan main terom...,1
1015,baliho red army kota malang simbol bangkit pki,0
1334,usus buntu bukan sebab tel biji jambu,0
2283,mahasiswa usir tumpang gelap,1
318,suhu panas kuwait leleh mobil lampu lalulintas,1
1394,220 anak palestina ungsi siap adopsi belawan m...,0


## Save verify_dataset_clean

Menyimpan dataset yang telah melalui tahap text_cleansing.

In [ ]:
df.to_csv('verify_dataset_clean.csv', index=False)